In [52]:
import itertools
from typing import Tuple, Union

import sympy

from milad import geometric, invariants

sympy.init_printing()
ORDER = 10
NUM_POINTS = 8

In [53]:
def create_array(indexable, shape: Union[int, Tuple[int]]):
    if isinstance(shape, int):
        shape = (shape,)

    components = sympy.MutableDenseNDimArray.zeros(*shape)

    ranges = [range(entry) for entry in shape]
    for indices in itertools.product(*ranges):
        components[indices] = indexable[indices]

    return components

In [54]:
r = sympy.IndexedBase('r')  # Positions of particles
w = sympy.IndexedBase('w')  # Weights of particles
m = sympy.IndexedBase('m')  # Weights of particles

moments = geometric.from_deltas_analytic(ORDER, NUM_POINTS, pos_symbols=r, weight_symbols=w)
print("Shape of m: {} ({})".format(moments.shape, type(moments)))

r_array = create_array(r, (NUM_POINTS, 3))
r_derivatives = sympy.derive_by_array(moments, r_array)

print("Shape of dm/dr: {}".format(r_derivatives.shape))

Shape of m: (11, 11, 11) (<class 'sympy.tensor.array.dense_ndim_array.MutableDenseNDimArray'>)
Shape of dm/dr: (8, 3, 11, 11, 11)


In [55]:
w_array = create_array(w, NUM_POINTS)
w_derivatives = sympy.derive_by_array(moments, w_array)

# print(w_derivatives.shape)

invs = invariants.read(read_max=10)
phi = sympy.Array(invs.apply(m))

print("Phi.shape: {}".format(phi.shape))

Phi.shape: (10,)


In [56]:
m_array = create_array(m, (ORDER + 1, ORDER + 1, ORDER + 1))

derivs = sympy.derive_by_array(phi, m_array)

print("Shape of dp/dm: {}".format(derivs.shape))


Shape of dp/dm: (11, 11, 11, 10)


In [57]:
def total_deriv(p, m_drivs, p_derivs):
    total = 0
    for entry in p.free_symbols:
        if isinstance(entry, sympy.Indexed):
            result = m_drivs[entry.indices] * p_derivs[entry.indices]
            total += result
    return total


for i in range(NUM_POINTS):
    for d in range(3):
        print(total_deriv(phi[3], r_derivatives[i, d], derivs[:,:,:,3]))




6*m[1, 0, 2]*r[0, 2]**2*w[0] + 12*m[1, 1, 1]*r[0, 1]*r[0, 2]*w[0] + 6*m[1, 2, 0]*r[0, 1]**2*w[0] + 12*m[2, 0, 1]*r[0, 0]*r[0, 2]*w[0] + 12*m[2, 1, 0]*r[0, 0]*r[0, 1]*w[0] + 6*m[3, 0, 0]*r[0, 0]**2*w[0]
6*m[0, 1, 2]*r[0, 2]**2*w[0] + 12*m[0, 2, 1]*r[0, 1]*r[0, 2]*w[0] + 6*m[0, 3, 0]*r[0, 1]**2*w[0] + 12*m[1, 1, 1]*r[0, 0]*r[0, 2]*w[0] + 12*m[1, 2, 0]*r[0, 0]*r[0, 1]*w[0] + 6*m[2, 1, 0]*r[0, 0]**2*w[0]
6*m[0, 0, 3]*r[0, 2]**2*w[0] + 12*m[0, 1, 2]*r[0, 1]*r[0, 2]*w[0] + 6*m[0, 2, 1]*r[0, 1]**2*w[0] + 12*m[1, 0, 2]*r[0, 0]*r[0, 2]*w[0] + 12*m[1, 1, 1]*r[0, 0]*r[0, 1]*w[0] + 6*m[2, 0, 1]*r[0, 0]**2*w[0]
6*m[1, 0, 2]*r[1, 2]**2*w[1] + 12*m[1, 1, 1]*r[1, 1]*r[1, 2]*w[1] + 6*m[1, 2, 0]*r[1, 1]**2*w[1] + 12*m[2, 0, 1]*r[1, 0]*r[1, 2]*w[1] + 12*m[2, 1, 0]*r[1, 0]*r[1, 1]*w[1] + 6*m[3, 0, 0]*r[1, 0]**2*w[1]
6*m[0, 1, 2]*r[1, 2]**2*w[1] + 12*m[0, 2, 1]*r[1, 1]*r[1, 2]*w[1] + 6*m[0, 3, 0]*r[1, 1]**2*w[1] + 12*m[1, 1, 1]*r[1, 0]*r[1, 2]*w[1] + 12*m[1, 2, 0]*r[1, 0]*r[1, 1]*w[1] + 6*m[2, 1, 0]*r[1, 0